<a href="https://colab.research.google.com/github/kike22-22/IA/blob/main/ProyectoIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ruta de las carpetas con las imágenes
DATA_DIR = 'Users\\enriquemonreal\\Desktop\\IA_EXAMEN'  # Carpeta que contiene las subcarpetas de las clases

# Definir tamaño de las imágenes (64x64 píxeles)
IMG_SIZE = (64, 64)

# Listas para almacenar las imágenes y las etiquetas
data = []  # Lista de imágenes procesadas
labels = []  # Lista de etiquetas asociadas

# Recorrer las subcarpetas correspondientes a las clases (Cuchillos y Armas)
for label, folder in enumerate(['Cuchillos', 'Armas']):
    class_dir = os.path.join(DATA_DIR, folder)  # Ruta de la subcarpeta
    for img_name in os.listdir(class_dir):  # Iterar sobre cada archivo en la carpeta
        img_path = os.path.join(class_dir, img_name)  # Ruta completa de la imagen
        img = cv2.imread(img_path)  # Leer la imagen
        if img is not None:  # Comprobar si la imagen se cargó correctamente
            img = cv2.resize(img, IMG_SIZE)  # Redimensionar la imagen al tamaño deseado
            img = img / 255.0  # Normalizar los valores de los píxeles a un rango de [0, 1]
            data.append(img)  # Agregar la imagen procesada a la lista
            labels.append(label)  # Etiquetar la imagen (0=Cuchillos, 1=Armas)

# Convertir las listas a arrays de NumPy para su uso con TensorFlow
data = np.array(data)  # Array de imágenes
labels = np.array(labels)  # Array de etiquetas

# Crear un generador de aumento de datos (data augmentation)
datagen = ImageDataGenerator(
    rotation_range=20,        # Rotaciones aleatorias en un rango de 20 grados
    width_shift_range=0.2,    # Desplazamiento horizontal en un 20% del ancho
    height_shift_range=0.2,   # Desplazamiento vertical en un 20% de la altura
    shear_range=0.2,          # Aplicar transformaciones de cizalladura
    zoom_range=0.2,           # Zoom aleatorio en un rango de 20%
    horizontal_flip=True,     # Volteo horizontal aleatorio
    fill_mode='nearest'       # Rellenar píxeles vacíos con el valor más cercano
)

# Ajustar el generador de datos al conjunto de imágenes
datagen.fit(data)

# Crear el modelo de red neuronal convolucional
model = models.Sequential([
    layers.InputLayer(input_shape=(64, 64, 3)),  # Entrada: imágenes de 64x64 con 3 canales (RGB)
    layers.Conv2D(32, (3, 3), activation='relu'),  # Capa convolucional con 32 filtros y activación ReLU
    layers.MaxPooling2D((2, 2)),  # Reducción de dimensionalidad con MaxPooling
    layers.Conv2D(64, (3, 3), activation='relu'),  # Segunda capa convolucional con 64 filtros
    layers.MaxPooling2D((2, 2)),  # Otra reducción de dimensionalidad
    layers.Flatten(),  # Aplanar las características extraídas para la capa totalmente conectada
    layers.Dense(64, activation='relu'),  # Capa densa intermedia con 64 neuronas
    layers.Dense(2, activation='softmax')  # Capa de salida con 2 clases (Cuchillos, Armas)
])

# Compilar el modelo con el optimizador Adam y la pérdida categórica esparsa
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo utilizando el generador de datos
model.fit(datagen.flow(data, labels, batch_size=32), epochs=10)

# Guardar el modelo entrenado en un archivo
model.save('deteccion_armas.h5')  # Se guarda como deteccion_armas.h5
print("Modelo entrenado y guardado como 'deteccion_armas.h5'.")

# Abrir la cámara para realizar predicciones en tiempo real
cap = cv2.VideoCapture(0)  # Iniciar la captura de video desde la cámara

while True:
    # Capturar un fotograma desde la cámara
    ret, frame = cap.read()
    if not ret:  # Verificar si la cámara devolvió una imagen
        print("Error: No se pudo capturar la imagen.")
        break

    # Redimensionar el fotograma capturado
    img = cv2.resize(frame, IMG_SIZE)

    # Normalizar el fotograma
    img = img / 255.0

    # Añadir una dimensión al fotograma para que sea compatible con el modelo
    img = np.expand_dims(img, axis=0)

    # Realizar la predicción con el modelo
    prediction = model.predict(img)

    # Obtener el índice de la clase con mayor probabilidad
    class_idx = np.argmax(prediction)

    # Definir los nombres de las clases
    class_names = ['Cuchillos', 'Armas']

    # Mostrar la predicción en la pantalla
    label = class_names[class_idx]
    confidence = prediction[0][class_idx] * 100  # Probabilidad asociada a la clase predicha
    cv2.putText(frame, f'{label} ({confidence:.2f}%)', (10, 30),  # Agregar texto a la imagen
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)

    # Mostrar el fotograma con la predicción
    cv2.imshow('Predicción en tiempo real', frame)

    # Salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar la cámara y cerrar todas las ventanas de OpenCV
cap.release()
cv2.destroyAllWindows()
